In [97]:
# Third Party libraries
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

from sklearn.linear_model import LinearRegression

In [98]:
# Import Data_cleaning module
from modules.data_clean import DataCleaner

# Load DF
df = pd.read_csv('../../data/raw/Fire_Incidents_Data.csv', low_memory=False)

In [99]:
# Cleanse Dataframe
df = DataCleaner.cleanse_dataframe(df)

### Defining all features
- Numerical Columns
- Categorial Columns
    - Ordinal Columns
    - Onehot Columns

In [100]:
NUMERICAL_COLS = [
    "Civilian_Casualties",
    "Count_of_Persons_Rescued",
    "Estimated_Dollar_Loss",
    "Estimated_Number_Of_Persons_Displaced",
    "Number_of_responding_apparatus",
    "Number_of_responding_personnel",
    "TFS_Firefighter_Casualties"
]

CATEGORICAL_COLS = ['Area_of_Origin', 'Building_Status', 'Business_Impact', 'Extent_Of_Fire', 'Final_Incident_Type', 
                'Fire_Alarm_System_Impact_on_Evacuation', 'Fire_Alarm_System_Operation', 'Fire_Alarm_System_Presence', 
                'Ignition_Source', 'Initial_CAD_Event_Type', 'Material_First_Ignited', 'Method_Of_Fire_Control', 
                'Possible_Cause', 'Property_Use', 'Smoke_Alarm_at_Fire_Origin', 'Smoke_Alarm_at_Fire_Origin_Alarm_Failure', 
                'Smoke_Alarm_at_Fire_Origin_Alarm_Type', 'Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation', 
                'Smoke_Spread', 'Sprinkler_System_Operation', 'Sprinkler_System_Presence', 'Status_of_Fire_On_Arrival', 'Incident_Ward']

# ['Business_Impact', 'Extent_Of_Fire', 'Sprinkler_System_Presence', 'Status_of_Fire_On_Arrival']
ORDINAL_COLS = CATEGORICAL_COLS[2:4] + CATEGORICAL_COLS[20:22]

# Everything else.
ONEHOT_COLS = list(set(CATEGORICAL_COLS) - set(ORDINAL_COLS))

### Imputation

#### Categorical Imputation

Will use SimpleImputer, as it can work with strings.

In [101]:
# Specify pd.NA as the missing value indicator for SimpleImputer
imputer = SimpleImputer(strategy='most_frequent', missing_values=pd.NA)

# Apply the imputer to the categorical columns
df[CATEGORICAL_COLS] = imputer.fit_transform(df[CATEGORICAL_COLS])

#### Numerical imputation.

This uses KNN for numerical columns missing data.

In [102]:
import numpy as np

# Replace pd.NA with np.nan
# KNNImputer does not directly support pd.NA
df = df.replace(pd.NA, np.nan)

imputer = KNNImputer()

# Apply the imputer to the categorical columns
df[NUMERICAL_COLS] = imputer.fit_transform(df[NUMERICAL_COLS])

### Categorical Encoding

Encoding all categorical columns as ordinal.

This yields bad results.

In [103]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import numpy as np

# Specify ordinal encoder
ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan)

# Apply the encoder to specified columns
df[CATEGORICAL_COLS] = ordinal_encoder.fit_transform(df[CATEGORICAL_COLS])
df

,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Extent_Of_Fire,Final_Incident_Type,Fire_Alarm_System_Impact_on_Evacuation,Fire_Alarm_System_Operation,Fire_Alarm_System_Presence,Ignition_Source,Incident_Ward,Initial_CAD_Event_Type,Material_First_Ignited,Method_Of_Fire_Control,Number_of_responding_apparatus,Number_of_responding_personnel,Possible_Cause,Property_Use,Smoke_Alarm_at_Fire_Origin,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Firefighter_Casualties
0,60.0,0.0,5.0,0.0,0.0,15000.0,0.2,0.0,0.0,5.0,0.0,0.0,83.0,1.0,126.0,25.0,0.0,1.0,4.0,22.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,5.0,0.0
1,56.0,0.0,5.0,0.0,0.0,50.0,0.0,0.0,0.0,5.0,0.0,0.0,83.0,18.0,62.0,51.0,0.0,1.0,4.0,2.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,1.0,0.0
3,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,3.0,83.0,5.0,60.0,52.0,0.0,6.0,22.0,22.0,138.0,8.0,9.0,5.0,5.0,9.0,5.0,3.0,2.0,0.0
5,60.0,0.0,5.0,0.0,0.0,1500.0,4.0,0.0,0.0,5.0,0.0,0.0,83.0,25.0,126.0,25.0,0.0,7.0,25.0,22.0,281.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,3.0,0.0
6,6.0,0.0,0.0,0.0,0.0,2000.0,0.0,3.0,0.0,5.0,2.0,2.0,47.0,19.0,72.0,5.0,0.0,6.0,22.0,6.0,71.0,0.0,9.0,4.0,4.0,1.0,5.0,2.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29419,60.0,0.0,5.0,0.0,0.0,50000.0,1.6,0.0,0.0,5.0,0.0,0.0,69.0,2.0,115.0,59.0,0.0,5.0,18.0,26.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,2.0,0.0
29420,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,83.0,13.0,26.0,59.0,0.0,5.0,17.0,28.0,81.0,3.0,9.0,3.0,2.0,2.0,2.0,0.0,1.0,0.0
29421,60.0,0.0,5.0,0.0,0.0,5000.0,0.0,0.0,0.0,5.0,0.0,0.0,68.0,20.0,114.0,59.0,0.0,1.0,4.0,26.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,2.0,0.0
29422,25.0,0.0,5.0,0.2,0.0,2000.0,0.0,3.0,0.0,2.0,0.0,0.0,83.0,13.0,26.0,61.0,0.0,8.0,26.0,27.0,81.0,3.0,9.0,3.0,5.0,3.0,1.0,0.0,1.0,0.0


Encoding onehot columns - Work in progress/Not working.

In [104]:


# onehot_encoder = OneHotEncoder(handle_unknown="ignore")
# # Apply the imputer to the categorical columns
# df_encoded= onehot_encoder.fit_transform(df[ONEHOT_COLS])

# df_encoded

# from sklearn.preprocessing import OneHotEncoder
# from sklearn.linear_model import LinearRegression

# # X is a numpy array with your features
# # y is the label array
# enc = OneHotEncoder(sparse=False)
# X_transform = enc.fit_transform(X)

# # apply your linear regression as you want
# model = LinearRegression()
# model.fit(X_transform, y)

# print("Mean squared error: %.2f" % np.mean((model.predict(X_transform) - y) ** 2))

### Standard scaler on numerical Data

Will scale all numerical columns.

In [105]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the DataFrame
df[NUMERICAL_COLS] = scaler.fit_transform(df[NUMERICAL_COLS])

### Log transform response variable.

In [106]:
# Log transform the Estimated_Dollar_Loss column
df['Estimated_Dollar_Loss'] = np.log(df['Estimated_Dollar_Loss'] + 1)  # Adding 1 to avoid log(0) issue

In [107]:
df

,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Extent_Of_Fire,Final_Incident_Type,Fire_Alarm_System_Impact_on_Evacuation,Fire_Alarm_System_Operation,Fire_Alarm_System_Presence,Ignition_Source,Incident_Ward,Initial_CAD_Event_Type,Material_First_Ignited,Method_Of_Fire_Control,Number_of_responding_apparatus,Number_of_responding_personnel,Possible_Cause,Property_Use,Smoke_Alarm_at_Fire_Origin,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Firefighter_Casualties
0,60.0,0.0,5.0,-0.223876,-0.052416,-0.154712,-0.120846,0.0,0.0,5.0,0.0,0.0,83.0,1.0,126.0,25.0,0.0,-0.888237,-0.922813,22.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,5.0,-0.09813
1,56.0,0.0,5.0,-0.223876,-0.052416,-0.382436,-0.122911,0.0,0.0,5.0,0.0,0.0,83.0,18.0,62.0,51.0,0.0,-0.888237,-0.922813,2.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,1.0,-0.09813
3,56.0,0.0,0.0,-0.223876,-0.052416,-0.383292,-0.122911,0.0,0.0,6.0,2.0,3.0,83.0,5.0,60.0,52.0,0.0,-0.214325,-0.141605,22.0,138.0,8.0,9.0,5.0,5.0,9.0,5.0,3.0,2.0,-0.09813
5,60.0,0.0,5.0,-0.223876,-0.052416,-0.357935,-0.081621,0.0,0.0,5.0,0.0,0.0,83.0,25.0,126.0,25.0,0.0,-0.079543,-0.011403,22.0,281.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,3.0,-0.09813
6,6.0,0.0,0.0,-0.223876,-0.052416,-0.349623,-0.122911,3.0,0.0,5.0,2.0,2.0,47.0,19.0,72.0,5.0,0.0,-0.214325,-0.141605,6.0,71.0,0.0,9.0,4.0,4.0,1.0,5.0,2.0,1.0,-0.09813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29419,60.0,0.0,5.0,-0.223876,-0.052416,0.235156,-0.106395,0.0,0.0,5.0,0.0,0.0,69.0,2.0,115.0,59.0,0.0,-0.349107,-0.315207,26.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,2.0,-0.09813
29420,23.0,0.0,0.0,-0.223876,-0.052416,-0.383292,-0.122911,0.0,0.0,2.0,0.0,0.0,83.0,13.0,26.0,59.0,0.0,-0.349107,-0.358607,28.0,81.0,3.0,9.0,3.0,2.0,2.0,2.0,0.0,1.0,-0.09813
29421,60.0,0.0,5.0,-0.223876,-0.052416,-0.301155,-0.122911,0.0,0.0,5.0,0.0,0.0,68.0,20.0,114.0,59.0,0.0,-0.888237,-0.922813,26.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,2.0,-0.09813
29422,25.0,0.0,5.0,0.312590,-0.052416,-0.349623,-0.122911,3.0,0.0,2.0,0.0,0.0,83.0,13.0,26.0,61.0,0.0,0.055240,0.031997,27.0,81.0,3.0,9.0,3.0,5.0,3.0,1.0,0.0,1.0,-0.09813


### Train test split

In [108]:
from sklearn.model_selection import train_test_split

# List all columns in the DataFrame
all_columns = df.columns.tolist()

# Use every other column in the df except for the response variable
features = [col for col in all_columns if col != 'Estimated_Dollar_Loss']

# Separate features from response variable
X, y = df[features], df['Estimated_Dollar_Loss']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=12)

### Apply logistic regressor (no pipeline)

#### Helper Function

In [109]:
# Function: Print Results of a Model (R2, MEAN ABSOLUTE ERROR, MEAN SQUARED ERROR, COEF and INTERCEPT)

def print_results(r2, mae, mse, coefficients, intercept):
    print(f"R-squared score: {r2:.4f}")
    print(f"Mean Absolute Error: {mae}")
    print(f"Mean Squared Error: {mse}")
    print(f"Coefficients: {coefficients}")
    print(f"Intercept: {intercept}")

# Function: Return Results (R2, MEAN ABSOLUTE ERROR, MEAN SQUARED ERROR, COEF and INTERCEPT)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
def get_results(model, y_test, y_pred):
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    coefficients = model.coef_
    intercept = model.intercept_
    return (r2, mae, mse, coefficients, intercept)

In [110]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

lr = LinearRegression() # to do - settings hyperparameters 
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)
residuals_lr = y_test - y_pred_lr

(r2_lr, mae_lr, mse_lr, coefficients_lr, intercept_lr) = get_results(lr, y_test, y_pred_lr)

# Results - Another option would be to use statsmodels to display a summary
print("---------MLR----------")
print_results(r2_lr, mae_lr, mse_lr, coefficients_lr, intercept_lr)
print("----------------------")

---------MLR----------
R-squared score: 0.4990
Mean Absolute Error: 0.18968047768279186
Mean Squared Error: 0.09110818627232162
Coefficients: [-5.40044435e-04 -4.03859967e-03  9.09590650e-03  1.80537518e-02
  3.77203230e-03  2.65446504e-02  3.52145359e-02  7.82953650e-02
  5.50008209e-03  1.77295533e-03  1.50912687e-02 -4.48423262e-05
 -1.15125856e-04  1.66088344e-03  2.75662755e-04 -2.00346053e-02
 -1.15764062e-01  3.13578599e-01  1.55675179e-03  2.14311327e-04
  4.58804521e-03  1.09614703e-02  2.10398828e-03 -2.58977509e-03
  3.60416043e-03  2.25637117e-03 -2.04753760e-02  3.19567953e-02
  2.17338313e-02]
Intercept: -0.5441390798336032
----------------------


### Create mini-pipeline
Applies the following:
- Ordinal encoding
- One hot encoding
- Imputes the data 
- Logistic regressor

In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Create a pipeline with ColumnTransformer, OneHotEncoder, and LogisticRegression
# pipeline = Pipeline([
#     ('preprocessor', ColumnTransformer([
#         ('ordinal', OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan), ORDINAL_COLS),
#         ('onehot', OneHotEncoder(handle_unknown='ignore'), ONEHOT_COLS),
#         ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA), ONEHOT_COLS + ORDINAL_COLS)
#     ], remainder='passthrough')),
#     ('classifier', LogisticRegression(max_iter=1000))
# ])


### Apply Logistic Regressor (pipeline)

In [ ]:

pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred = pipeline.predict(X_test)

# Evaluate the model (e.g., using accuracy score)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)